In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install haversine

In [ ]:
import requests
import json
import pandas
from haversine import haversine
import numpy as np

In [ ]:
#직선거리 가까운 10개만
def distance_top10():
    building_lon_lat = pandas.read_excel('/content/drive/MyDrive/데이터톤/data/data/교내 랜드마크 위도 경도.xlsx')

    current_lon = 127.02727228835887
    current_lat = 37.58155881472965

    ls = []
    df_final = pandas.DataFrame(columns=['building', 'distance'])

    for idx,row in building_lon_lat.iterrows():
        start = (float(current_lat), float(current_lon))
        goal = (float(row['lon,lat'].split(',')[1]),float(row['lon,lat'].split(',')[0]))
        ls.append((row['building'],haversine(start,goal),row['lon,lat']))
        ls = sorted(ls, key = lambda x: x[1], reverse = False)[:10]

    for row in ls:
        request_str = 'http://router.project-osrm.org/route/v1/foot/'+str(current_lat)+','+str(current_lat)+';'+row[2]+'?overview=false'
        
        r = requests.get(request_str)
        routes = json.loads(r.content)
        distance = routes.get("routes")[0]['distance']
        df_final.loc[len(df_final)] = [row[0],distance]

    df_final.sort_values('distance', inplace=True)
    return df_final

In [ ]:
# 직선거리 400미터 이하만 
def distance_under400():
    building_lon_lat = pandas.read_excel('/content/drive/MyDrive/데이터톤/data/data/교내 랜드마크 위도 경도.xlsx')

    current_lon = 127.02727228835887
    current_lat = 37.58155881472965

    df_final = pandas.DataFrame(columns=['building', 'distance'])

    for idx, row in building_lon_lat.iterrows():
        start = (float(current_lat), float(current_lon))
        goal = (float(row['lon,lat'].split(',')[1]),float(row['lon,lat'].split(',')[0]))

        if haversine(start, goal) < 0.4:
            request_str = 'http://router.project-osrm.org/route/v1/foot/'+str(current_lat)+','+str(current_lat)+';'+row['lon,lat']+'?overview=false'
            
            r = requests.get(request_str)
            routes = json.loads(r.content)
            distance = routes.get("routes")[0]['distance']
            df_final.loc[len(df_final)] = [row['building'],distance]

    df_final.sort_values('distance', inplace=True)
    return df_final

In [ ]:
building_lon_lat = pandas.read_excel('/content/drive/MyDrive/데이터톤/data/data/교내 랜드마크 위도 경도.xlsx')

request_str='http://router.project-osrm.org/table/v1/foot/'

for idx, row in building_lon_lat.iterrows():
    request_str+=(row['lon,lat']+';')

r = requests.get(request_str[:-1])
table = json.loads(r.content)
table_1 = table['durations']
table_2 = np.array(table_1).reshape(-1,98)

table_2

array([[  0. ,  38. ,  97.2, ..., 155.4, 154.1, 155.4],
       [ 38. ,   0. ,  79.1, ..., 123.4, 122.1, 123.4],
       [ 92.7,  72.7,   0. , ..., 128.9, 127.6, 128.9],
       ...,
       [146.7, 119.3, 131.3, ...,   0. ,  74.7,   0. ],
       [146. , 118.6, 130.6, ...,  75.7,   0. ,  75.7],
       [146.7, 119.3, 131.3, ...,   0. ,  74.7,   0. ]])

In [ ]:
lst = building_lon_lat['building']

In [ ]:
from google.colab import files
df_array = pandas.DataFrame(table_2, columns=lst) 

df_array.to_csv('건물별 이동시간 추가.csv', index = None,encoding='euc-kr') 
files.download('건물별 이동시간 추가.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>